In [1]:
import pandas as pd

ModuleNotFoundError: No module named 'airflow'

In [ ]:
'''
#Da um erro referente a configuracoes de INSTALLED_APPS, tentei criar o arquivo apps.py porem ainda assim retorna erro... precisa estudar melhor django para aplicar o ORM nao eh tao simples como pandas

from django.db import models
from django.apps import apps
import apps as app

apps.get_app_config('admin').verbose_name

class Student(models.Model):
    name = models.CharField(max_length=50)
    grade = models.IntegerField()

class Club(models.Model):
    name = models.CharField(max_length=50)
    members = models.ManyToManyField(Student)
'''

In [2]:
#Aqui atraves do parametro index_col estipula-se a coluna que sera o indice do dataframe
amzn  = pd.read_csv('data/amzn_june_2020.csv', sep=';', index_col=0)
msft  = pd.read_csv('data/msft_june_2020.csv', sep=';', index_col=0)
googl = pd.read_csv('data/google_june_2020.csv', sep=';', index_col=0)

#Test sem index
amzn_noidx  = pd.read_csv('data/amzn_june_2020.csv', sep=';')

In [3]:
'''
Utilizei essa funcao pois a coluna vinha como Unnamed antes de colocar o parametro index_col 
e indica-lá como a o index do DataFrame, resultado é que a coluna não aparece mais no dataFrame
executar abaixo o comando para o amzn_noidx para ver a diferença
'''
amzn_noidx.rename(columns={"Unnamed: 0" : "Index", 'Date' : 'Data_Full'}, inplace=True)
amzn_noidx.head()

,Index,Data_Full,Close,Volume,Symbol
0,0,01/06/2020,24.710.400.390.625,2928900,AMZN
1,1,02/06/2020,2.472.409.912.109.370,2529900,AMZN
2,2,03/06/2020,247.839.990.234.375,2671000,AMZN
3,3,04/06/2020,246.060.009.765.625,2948700,AMZN
4,4,05/06/2020,2483.0,3306400,AMZN


In [ ]:
amzn.nlargest(10, columns='Volume').tail(3)

In [ ]:
#Exemplo de renomear as colunas pelo index (posicao) delas no dataframe
amzn_noidx.rename(columns={amzn_noidx.columns[0]: 'Data_Full', amzn_noidx.columns[1]: 'Fechamento'}, inplace=True)
amzn_noidx.head()

In [ ]:
amzn.head(1)

In [ ]:
msft.head(1)

In [ ]:
googl.head(1)

-Verificar as info gerais do DataFrame, verificando que ambos tem as mesmas estruturas e conjuntos de registros

In [ ]:
googl.info()

# Concatenation of Tidy Data Source

In [ ]:
all_df = pd.DataFrame()
all_df = all_df.append([amzn, googl, msft])

In [ ]:
all_df.info()

In [ ]:
all_df.reset_index(drop=True, inplace=True)

In [ ]:
all_df.shape

In [ ]:
all_df.head(3)

# Concatenation of tidy data source - pd.concat()

In [ ]:
#concat e append são a mesma funcionalidade -->> adicionar linhas ao final do df, porem o concat eh mais performatico e recente

all_concat = pd.concat([amzn, googl, msft])
all_concat.reset_index(drop=True, inplace=True)
all_concat.shape


In [ ]:
all_concat.head(3)

# Multi-Level indexing

In [ ]:
all_concat.set_index(['Symbol', 'Date'], inplace=True)
all_concat.head(3)

In [ ]:
all_concat.sort_index(inplace=True)
all_concat.head()

In [ ]:
#Por usar multi index infere que os campos ficam acessiveis como tuples, aonde tem que passar pelo loc, qual a chave e as demais colunas
#todas as colunas caracterizada pelo ':' para a tupla AMZN em 02/06/2020
all_concat.loc[('AMZN', '02/06/2020'), :]

In [ ]:
#Somente a coluna Volume
all_concat.loc[('AMZN', '01/06/2020'), 'Volume']

In [ ]:
#TUDO das tuplas somente para Volume
all_concat.loc[:, 'Volume']

In [ ]:
all_concat.index.levels

In [ ]:
all_concat.index.levels[0]

In [ ]:
all_concat.index.levels[1]

In [ ]:
#Possivel passar lista interna para o Index como abaixo, e retornar os valores dentro dessa lista
all_concat.loc[(['AMZN', 'GOOGL'], '02/06/2020'), :]

In [ ]:
#Comando slice -->> Pesquisar mais nunca usei
all_concat.loc[(slice(None), '02/06/2020'), :]

In [ ]:
all_concat.loc[(slice('AMZN','GOOGL',3), '02/06/2020'), :]

In [ ]:
all_concat.head()

# Merging and concatenating tidy data

In [ ]:
# duplica todas as colunas
all_concat_columns = pd.concat([amzn, googl, msft], axis=1)

In [ ]:
all_concat_columns.head()

In [ ]:
'''
DataFrame acima concatenado pelas colunas é poluído com informações duplicadas, e colunas não identificaveis como por exemplo Close e Volume, não tem relacao direta com o index externo
Podendo levar a erros fatais de integridade de dados, utilizando o parametro verify_integrity, garante essa verificacao
Se for pra fazer o append por linhas, usar o pandas.concat(), se for por colunas, usar o pandas.merge() existem diferenças cruciais sobre a estrutura e principalmente argumentos
concat() é mais performatico que o append() ambos tem a mesma função que é realizar o merge por linhas

#Lanca um erro
pd.concat([amzn, googl, msft], axis=1, verify_integrity=True)

'''

In [ ]:
'''
Essa funcao pega o primeiro valor da coluna Symbol, e remove todos as duplicatas, nesse caso todos os valores
ja que o dataframe passado tem o nome da empresa como symbol, entao é removido também a coluna symbol,
em seguida define a data como index e renomeia as colunas atraves da atribuicao de lista 
por concatenar o valor de symbol com o nome atual do valor da coluna do dataframe passado
'''

def new_agg_df(df):

    my_list = []
    symbol = df['Symbol'].drop_duplicates().values[0]
    df.drop(['Symbol'], axis=1, inplace=True)
    df.set_index('Date', inplace=True)
    my_list = [symbol+'_'+elem for elem in df.columns]
    df.columns = my_list
    return df

In [ ]:
#Chama a funcao passando o dataframe original
amzn_agg = new_agg_df(amzn)
msft_agg = new_agg_df(msft)
googl_agg = new_agg_df(googl)

In [ ]:
#Temos o dataframe com a identificao de cada coluna individual atrelada ao dataframe original
googl_agg.head(1)

In [ ]:
#Resetando o index dos dataframes, através do argumento on, especifica-se qual sera o novo index que é compartilhado por ambos os dataframes
#how é a clausula join pesquisar se sao os mesmos do SQL (inner, left, right, ...)
#é possível também especificar diferentes index para cada dataframe right_on, left_on
#quando nao se reseta o index, podemos estipular qual index vai se manter atraves somente do argumento left=True, right=True
amzn_agg.reset_index().merge(msft_agg.reset_index(), how='inner', on='Date').head()

In [ ]:
all_merge_df = amzn_agg.merge(msft_agg, left_index=True, right_index=True).merge(googl_agg, left_index=True, right_index=True)

all_merge_df.head()

# Transformation of a dataset

In [ ]:
#Todos os dataframes concatenados por linhas, sem index
all_df.head()

## The pivot method

In [ ]:
'''
O index é a coluna que sera o index do dataframe pivot, a coluna, significa a coluna do dataframe no qual o pivot sera executado e mostrar os valores
e values, e qual a coluna do df original, que devera ser mostrado como valores da coluna estipulada no columns para o pivot
'''
pivot_close = all_df.pivot(index='Date', columns='Symbol', values='Close').sort_index()

In [ ]:
pivot_close.head()

In [ ]:
pivot_close_volume = all_df.pivot(index='Date', columns='Symbol').sort_index()

In [ ]:
pivot_close_volume.head()

In [ ]:
#Pesquisar diferença entre unstack e pivot -> Segundo aula quando tiver multi-index dataframe é melhor utilizar o unstack
another_df = all_df.set_index(['Symbol', 'Date'])
unstacked_df = another_df.unstack(level='Symbol')
unstacked_df.head()

In [ ]:
unstacked_df.stack(level='Symbol').head()

In [ ]:
stack_df = unstacked_df.stack(level='Symbol')
swapped_df = stack_df.swaplevel('Symbol', 'Date')
swapped_df.head()

In [ ]:
swapped_df.sort_index().head()

# Melting with Pandas

In [ ]:
all_df.shape

In [ ]:
#Pesquisar melhor o Melt
all_df.melt(id_vars=['Symbol', 'Date'])

## Aggregating results with Pandas

In [ ]:
all_df.head(1)

In [ ]:
# ValueError: Unable to parse string "24.710.400.390.625" at position 0
all_df['Close'] = pd.to_numeric(all_df['Close'])

In [ ]:
pivot_close = all_df.pivot(
    index='Symbol', 
    columns='Date', 
    values='Volume'
).sort_index().reset_index()

mean_pivot = pd.DataFrame()
mean_pivot['Symbol'] = pivot_close['Symbol'].to_list()

In [ ]:
#Por alguma razão esta dando erro de conversão, tentei separadamente aplicar o dtypes antes no all_df mas deu erro mesmo assim, pesquisar depois o erro

#Erro esta na formatacao do numero em Close, pois ao substituir por Volume funcionou

mean_pivot['average_price'] = pivot_close.set_index('Symbol').apply(
    lambda x: x.mean(), axis=1
).to_list()

In [ ]:
mean_pivot.set_index('Symbol')

In [ ]:
mean_pivot.head()

In [ ]:
#Por alguma razao os valores na coluna Close nos arquivos FF estao dando problema, pois ao trocar para volume funcionou
all_df.pivot_table(
    values='Volume',
    index='Symbol',
    aggfunc='mean',
)

In [ ]:
all_df.pivot_table(
    index='Symbol',
    aggfunc=['mean','median','max','min','std'],
)

## Group by

In [ ]:
#a coluna close nao funciona para as funcoes que tratam numeros, por isso erro no mean e median
grouped_stocks = all_df.groupby('Symbol')['Close'].max()
grouped_stocks

#Saida e um Series, usar o to_frame para transformar em um dataframe

In [ ]:
grouped_stocks.to_frame()

In [ ]:
all_df.groupby('Symbol')['Close'].agg(['max', 'min',])

In [ ]:
#possivel tambem utilizar funcoes de agregacao por cada coluna por um tipo diferente de agregacao
all_df.groupby('Symbol').agg({'Close' : ['max', 'min',], 'Volume' : ['median', 'max']})

# Plotting results with Pandas

In [ ]:
import matplotlib.pyplot as plt